# LangChain - Long term memory

## Install dependencies

In [1]:
!pip install langchain
!pip install langchain_openai
!pip install langchainhub
!pip install openai
!pip install python-dotenv

## Load environment variables

Change the path if necessary (read `README.md` first and follow the instructions to setup the `.env` file):

In [2]:
import os
os.chdir("/home/jovyan/work/")

In [3]:
%load_ext dotenv
%dotenv

## Utils

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain import hub

merge_prompt = hub.pull("cristobal/merge-human-facts-yaml")
merge_model = ChatOpenAI(model="gpt-4")
merge_output_parser = StrOutputParser()

merge_chain = merge_prompt | merge_model | merge_output_parser

In [5]:
result = merge_chain.invoke({"yaml": "name: unknown", "facts": "['User name is Cristóbal', 'User's age is 43']"})
print(result)

name: Cristóbal
age: 43


## Tools

In [6]:
from typing import List

from langchain_core.tools import tool

# Initial profile
user_facts = """---
Name: Cristóbal
Brother: 
  - Pablo
"""

@tool
def save_user_info(facts: List[str]):
    """Save relevant information that the user is sharing with you.
    Not only personal information, but also information relevant to his life that the user mentions in passing: the name of his pet, where he lives,...
    Examples:
    
    "My name is Frank and I'm 24 years old" -> ["User name is Frank", "User's age is 24"]
    "The other day I met my brother Paul, and we went to lunch" -> ["User has a brother called Paul"]
    "My work at LangChain is a lot of fun" -> ["User works at LangChain"]
    """
    print("- - - - - - - - - -")
    print(facts)
    global user_facts
    user_facts = merge_chain.invoke({"yaml": user_facts, "facts": facts})

tools = [save_user_info]

## Agent

In [7]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI

prompt = hub.pull("cristobal/openai-functions-agent-human-profile")

llm = ChatOpenAI(model="gpt-4", temperature=0)

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

## Tests

In [8]:
#from langchain.memory import ChatMessageHistory

#history = ChatMessageHistory()

def chat(input: str) -> str:
    response = agent_executor.invoke({"input": input, "human_profile": user_facts, "chat_history": []})
    #response = agent_executor.invoke({"input": input, "human_profile": user_facts, "chat_history": history.messages})
    #history.add_user_message(input)
    #history.add_ai_message(response["output"])
    return response["output"]

In [9]:
chat("Hi! Do you remember my name?")

'Yes, your name is Cristóbal. How can I assist you today?'

In [10]:
chat("Did I told you that I have 2 brothers and 2 sisters?")

- - - - - - - - - -
['User has 2 brothers', 'User has 2 sisters']


"No, you didn't mention that before. You only told me about your brother Pablo. It's nice to know you have a big family. Can you tell me more about your other siblings?"

In [11]:
chat("Muse, my favorite group, is going to play in Málaga, my city.")

- - - - - - - - - -
["User's favorite group is Muse", 'User lives in Málaga']


"That's great, Cristóbal! It must be exciting to have your favorite band, Muse, perform in your city, Málaga. Do you plan on going to the concert?"

In [12]:
print(user_facts)

---
Name: Cristóbal
Brother: 
  - Pablo
  - Unknown
Sister: 
  - Unknown
  - Unknown
Favorite Group: Muse
Location: Málaga


In [13]:
chat("Did I told you anything about my brothers and sisters?")

"Yes, Cristóbal, you mentioned that you have one brother named Pablo. You didn't provide the names of your other brother and sisters."